<h1> This is a further "experimental" EDA analysis of the "merged.csv" file. </h1>

Before running into conclusions it would be nice to mess with the data a bit more and see how they respond with each other. 

First, let's load the file:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore') # Bad practice my ass

merged = pd.read_csv('data/merged.csv')

<h2> sort it by ascending order regarding "logerror" feature </h2>


In [ ]:
# sort merged by ascending "logerror"
merged.sort_values(by='logerror', inplace=True)

<h2> And define a funtion that plots a simple scatter between a feature and "logerror"</h2>

With a first look we can see that in the relation `"area_live_finished - logerror"`, as the "area_live_finished" increases alot then the logerror converges to 0

In [ ]:
def plot_feature_logerror(feature, df=merged):
    """This function plots a scatter x-y style with the feature in xAxis and logerror in yAxis."""
    def is_numerical():
        return df[feature].dtype in ['float64', 'int64']
    def is_label():
        return feature in ['logerror', 'ID']
    def is_constant():
        return df[feature].max() == df[feature].min()
    
    if not (is_numerical() and not is_label() and not is_constant()):
        return None
    plt.scatter(df[feature].values, df["logerror"].values)
    plt.xlabel(feature); plt.ylabel("logerror"); plt.title(feature + " - logerror")
    plt.show()

# The lines bellow plot all features that can be plotted.   
[plot_feature_logerror(feature) for feature in merged.columns]

<h2> Find all non numerical columns of merged and convert them to numerical </h2>


In [ ]:
def is_numerical(feature, df=merged):
        if df[feature].dtype in ['float64', 'int64']:
            return True
        else:
            return False
nonNumericalFeatures = [col for col in merged.columns if is_numerical(col) == False]
print (nonNumericalFeatures)

<h2> So we find out the non numerical values! </h2>

These are: 
['flag_tub', 'zoning_landuse_county', 'zoning_property', 'flag_fireplace', 'tax_delinquency', 'transactiondate']
Let's investigate more what these values are:<br />
<b>flag_tub</b>              --> only "True" and NaN. <br />
<b>zoning_landuse_county</b> --> values like: 0100, 010C, 96, ...<br />
<b>zoning_property</b>       --> values like: 1NR1*, AH RM-CD*, WVRPD4OOOO, ...<br />
<b>flag_fireplace</b>        --> only "True" and NaN.<br />
<b>tax_delinquency</b>       --> only "Y" and NaN<br />
<b>transactiondate</b>       --> date from "2016-01-02" to "2016-12-30" <br />

<h2> So... </h2>
These features need a special treatment. We could just change them with a correspondence of integers (e.g. True --> 1 and NaN --> 0, or 1NR1* --> 1, WVRPD4OOOO --> 2 etc...) using a <b>dict</b>. 
Although, this may not seem alright because the data "won't make sense". For example, the mean value of these features is an irrelevant number (imagine that you have values ranging from 0 to 10 and the mean value is 5.5???). Therefore, a different approach may occur